In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import yfinance as yf
import pandas as pd
import os
import requests
import datetime as dt
import urllib.request
import json

# Coletando dados ESG

In [ ]:
# Leitura dos tickers das empresas S&P500.
sp_500 = pd.read_csv("sp_500_stocks.csv")
sp_500_sample = pd.read_csv("sp_500_stocks_sample.csv")

# Yahoo API URL
# adicionar ?={TICKER} no final da URL para consultar índices ESG da Sustainlytics
url = "https://query2.finance.yahoo.com/v1/finance/esgChart"


# Define filtro de data (valores inclusivos) dos dados ESG coletados pela API do Yahoo Finance
data_inicio = '2016-01-01' #YYYY-MM-DD
data_final = '2019-11-01' #YYYY-MM-DD
esgData = pd.DataFrame()
# Armazena os dados coletados do Yahoo Finance no dataframe esgData
for symbol in sp_500["Ticker"]:
  url = "https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={stock}".format(stock=symbol)
  connection = urllib.request.urlopen(url)
  data = connection.read()
  data_2 = json.loads(data)
  try:
    Formatdata = data_2["esgChart"]["result"][0]["symbolSeries"]
  except KeyError:
    print("Erro ao coletar dados de "+symbol+". API retornou nulo?")
    continue
  Formatdata_2 = pd.DataFrame(Formatdata)
  Formatdata_2["timestamp"] = pd.to_datetime(Formatdata_2["timestamp"], unit="s")
  Formatdata_2['ticker'] = symbol
  start_date = pd.to_datetime(data_inicio)
  end_date = pd.to_datetime(data_final)
  Formatdata_2.dropna(inplace=True)
  filtered_df = Formatdata_2[(Formatdata_2['timestamp'] >= start_date) & (Formatdata_2['timestamp'] <= end_date)]
  esgData = pd.concat([esgData, filtered_df], ignore_index=True)

esgData

Erro ao coletar dados de ABMD. API retornou nulo?
Erro ao coletar dados de ALGN. API retornou nulo?
Erro ao coletar dados de ALK. API retornou nulo?
Erro ao coletar dados de AMCR. API retornou nulo?
Erro ao coletar dados de AMD. API retornou nulo?
Erro ao coletar dados de BF.B. API retornou nulo?
Erro ao coletar dados de BKR. API retornou nulo?
Erro ao coletar dados de BRK.B. API retornou nulo?
Erro ao coletar dados de CARR. API retornou nulo?
Erro ao coletar dados de CTVA. API retornou nulo?
Erro ao coletar dados de DISCK. API retornou nulo?
Erro ao coletar dados de DOW. API retornou nulo?
Erro ao coletar dados de EVRG. API retornou nulo?
Erro ao coletar dados de FANG. API retornou nulo?
Erro ao coletar dados de FLIR. API retornou nulo?
Erro ao coletar dados de FOX. API retornou nulo?
Erro ao coletar dados de FTI. API retornou nulo?
Erro ao coletar dados de GOOG. API retornou nulo?
Erro ao coletar dados de HRB. API retornou nulo?
Erro ao coletar dados de HWM. API retornou nulo?
Erro a

,timestamp,esgScore,governanceScore,environmentScore,socialScore,ticker
0,2016-01-01,70.50,60.56,82.08,70.69,A
1,2016-02-01,70.45,60.41,82.08,70.69,A
2,2016-03-01,70.45,60.41,82.08,70.69,A
3,2016-04-01,71.96,60.42,82.08,74.03,A
4,2016-05-01,71.96,60.42,82.08,74.03,A
...,...,...,...,...,...,...
21197,2019-07-01,50.35,59.91,54.82,41.50,ZTS
21198,2019-08-01,50.35,59.91,54.82,41.50,ZTS
21199,2019-09-01,50.35,59.91,54.82,41.50,ZTS
21200,2019-10-01,50.31,59.76,54.82,41.50,ZTS


# Selecionando empresas com melhores índices ESG

In [ ]:
#print("Selecione a data para recuperar dados ESG (formato Y#YY-MM-DD)")
#esg_date = input()
dates = []
for i in range(1, 13):
  dates.append(f"2019-{i if i > 9 else '0'+str(i)}-01")

esg_data = []
for date in dates:
  esgData_filtro_data = esgData[esgData['timestamp'] == date]
  top_5_esg_companies = esgData_filtro_data.sort_values(by='esgScore', ascending=False).head(3)
  lst = top_5_esg_companies['ticker'].tolist()
  lst.insert(0, date)
  esg_data.append(lst)

esg_data

[['2019-01-01', 'HPQ', 'INTC', 'PLD'],
 ['2019-02-01', 'INTC', 'HPQ', 'PLD'],
 ['2019-03-01', 'INTC', 'HPQ', 'NVDA'],
 ['2019-04-01', 'INTC', 'HPQ', 'NVDA'],
 ['2019-05-01', 'INTC', 'HPQ', 'NVDA'],
 ['2019-06-01', 'INTC', 'HPQ', 'NVDA'],
 ['2019-07-01', 'INTC', 'HPQ', 'NVDA'],
 ['2019-08-01', 'HPQ', 'INTC', 'NVDA'],
 ['2019-09-01', 'HPQ', 'INTC', 'NVDA'],
 ['2019-10-01', 'HPQ', 'INTC', 'NVDA'],
 ['2019-11-01', 'HPQ', 'INTC', 'ES'],
 ['2019-12-01']]

In [ ]:
# Digite o ticker do asset que deseja analizar no modelo
print("Digite o ticker do asset que deseja analizar utilizando Random Forest")
ticker = input()

asset = yf.Ticker(ticker)

Digite o ticker do asset que deseja analizar utilizando Random Forest
INTC


In [ ]:
asset = asset.history(period="max")

In [ ]:
asset = asset.loc["2017-01-01":"2019-01-31"].copy()
#del asset["Dividends"]
del asset["Stock Splits"]

In [ ]:
#asset.plot.line(y='Close', use_index=True)

In [ ]:
asset["Tomorrow"] = asset["Close"].shift(-1)
asset["Target"] = (asset["Tomorrow"] > asset["Close"]).astype(int)
asset

,Open,High,Low,Close,Volume,Dividends,Tomorrow,Target
Date,,,,,,,,
2017-01-03 00:00:00-05:00,30.387815,30.653428,30.105601,30.379513,20196500,0.0,30.221809,0
2017-01-04 00:00:00-05:00,30.470821,30.520625,30.163707,30.221809,15915700,0.0,30.172005,0
2017-01-05 00:00:00-05:00,30.255011,30.479122,30.138805,30.172005,13986000,0.0,30.279911,1
2017-01-06 00:00:00-05:00,30.371216,30.445920,30.039198,30.279911,15114000,0.0,30.387825,1
2017-01-09 00:00:00-05:00,30.279919,30.620236,30.279919,30.387825,19461400,0.0,30.329718,0
...,...,...,...,...,...,...,...,...
2019-01-25 00:00:00-05:00,40.994462,41.501972,40.031947,41.160717,86455700,0.0,40.871952,0
2019-01-28 00:00:00-05:00,40.364445,40.889453,40.294442,40.871952,35928700,0.0,40.723206,0
2019-01-29 00:00:00-05:00,40.933207,41.099462,40.539453,40.723206,20988300,0.0,41.598221,1


In [ ]:
asset = asset.reset_index()
date_columns = asset.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    asset[date_column] = asset[date_column].dt.date
asset.to_excel('s&p.xlsx')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
"""
def predict(train, test, predictors, model):
  model.fit(train[predictors], train["Target"])
  preds = model.predict(test[predictors])
  preds = pd.Series(preds, index=test.index, name="Predictions")
  combined = pd.concat([test["Target"], preds], axis=1)
  return combined
"""

'\ndef predict(train, test, predictors, model):\n  model.fit(train[predictors], train["Target"])\n  preds = model.predict(test[predictors])\n  preds = pd.Series(preds, index=test.index, name="Predictions")\n  combined = pd.concat([test["Target"], preds], axis=1)\n  return combined\n'

In [ ]:
def backtest(data, model, predictors, start=1500, step=250):
  all_predictions = []

  for i in range(start, data.shape[0], step):
    train = data.iloc[0:i].copy()
    test = data.iloc[i:(i+step)].copy()
    predictions = predict(train, test, predictors, model)
    all_predictions.append(predictions)

  return pd.concat(all_predictions)

In [ ]:
# Últimos x dias
time_frames = [2, 5, 60, 250, 1000]
predictors = []

for time_frame in time_frames:
  rolling_averages = asset.rolling(time_frame).mean()
  ratio_column = f"Close_Ratio_{time_frame}_days"
  # Proporção preço de fechamento de hoje : preço de fechamento dos últimos x dias
  asset[ratio_column] = asset["Close"] / rolling_averages["Close"]

  trend_column = f"Trend_{time_frame}_days"
  # Verifica quantas vezes a ação aumentou nos últimos x dias
  asset[trend_column] = asset.shift(1).rolling(time_frame).sum()["Target"]

  predictors += [ratio_column, trend_column]

<ipython-input-15-5d0d37c4bebd>:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Date'], dtype='object')
  rolling_averages = asset.rolling(time_frame).mean()
<ipython-input-15-5d0d37c4bebd>:13: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Date'], dtype='object')
  asset[trend_column] = asset.shift(1).rolling(time_frame).sum()["Target"]
<ipython-input-15-5d0d37c4bebd>:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Date'], dtype='object')
  rolling_averages = asset.rolling(time_frame).mean()


In [ ]:
# Remove linhas que não possuem dados suficientes para calcular proporções e trends
#asset = asset.dropna()

In [ ]:
asset

,Date,Open,High,Low,Close,Volume,Dividends,Tomorrow,Target,Close_Ratio_2_days,Trend_2_days,Close_Ratio_5_days,Trend_5_days,Close_Ratio_60_days,Trend_60_days,Close_Ratio_250_days,Trend_250_days,Close_Ratio_1000_days,Trend_1000_days
0,2017-01-03,30.387815,30.653428,30.105601,30.379513,20196500,0.0,30.221809,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,30.470821,30.520625,30.163707,30.221809,15915700,0.0,30.172005,0,0.997398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-05,30.255011,30.479122,30.138805,30.172005,13986000,0.0,30.279911,1,0.999175,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-06,30.371216,30.445920,30.039198,30.279911,15114000,0.0,30.387825,1,1.001785,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-09,30.279919,30.620236,30.279919,30.387825,19461400,0.0,30.329718,0,1.001779,2.0,1.003289,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,2019-01-25,40.994462,41.501972,40.031947,41.160717,86455700,0.0,40.871952,0,0.971901,1.0,0.971098,2.0,0.990341,34.0,0.964119,127.0,NaN,NaN
519,2019-01-28,40.364445,40.889453,40.294442,40.871952,35928700,0.0,40.723206,0,0.996480,0.0,0.974261,1.0,0.982843,34.0,0.957516,127.0,NaN,NaN
520,2019-01-29,40.933207,41.099462,40.539453,40.723206,20988300,0.0,41.598221,1,0.998177,0.0,0.977772,1.0,0.979583,33.0,0.954114,127.0,NaN,NaN
521,2019-01-30,40.845707,41.720722,40.644454,41.598221,26931700,0.0,41.230713,0,1.010629,1.0,1.000463,2.0,1.000297,34.0,0.974568,128.0,NaN,NaN


In [ ]:
asset = asset.dropna()
asset

,Date,Open,High,Low,Close,Volume,Dividends,Tomorrow,Target,Close_Ratio_2_days,Trend_2_days,Close_Ratio_5_days,Trend_5_days,Close_Ratio_60_days,Trend_60_days,Close_Ratio_250_days,Trend_250_days,Close_Ratio_1000_days,Trend_1000_days


In [ ]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50)

In [ ]:
def predict(train, test, predictors, model):
  model.fit(train[predictors], train["Target"])
  preds = model.predict_proba(test[predictors])[:,1]
  preds[preds >= .55] = 1
  preds[preds < .55] = 0
  preds = pd.Series(preds, index=test.index, name="Predictions")
  combined = pd.concat([test["Target"], test["Close"], preds], axis=1)
  return combined

In [ ]:
predictions = backtest(asset, model, predictors)

ValueError: ignored

In [ ]:
predictions["Predictions"].value_counts()

In [ ]:
from sklearn.metrics import precision_score
precision_score(predictions["Target"], predictions["Predictions"])

In [ ]:
predictions['Asset'] = ticker
predictions

In [ ]:
# Transforma date index em coluna
predictions = predictions.reset_index()

In [ ]:
predictions

In [ ]:
date_columns = predictions.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    predictions[date_column] = predictions[date_column].dt.date

predictions.to_excel(f'{ticker}_output.xlsx')